In [65]:
import gradio as gr
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama.chat_models import ChatOllama
import fitz

In [66]:
# Variable global para almacenar las líneas extraídas del PDF
pdf_lines = []

In [67]:
# Función para extraer las líneas del PDF
def extract_lines_from_pdf_pymupdf(pdf_file):
    import fitz  # PyMuPDF
    lines = []
    with fitz.open(pdf_file.name) as doc:
        for page in doc:
            text = page.get_text("text")  # Obtener texto como líneas completas
            page_lines = text.splitlines()
            lines.extend([line.strip() for line in page_lines if line.strip()])
    return lines

In [68]:
# Función para realizar la búsqueda con Chroma y responder preguntas
def search_chroma(input, pdf_file=None):
    global pdf_lines  # Usamos la variable global pdf_lines
    
    try:
        if not pdf_lines:  # Si pdf_lines está vacío, procesamos el archivo
            pdf_lines = extract_lines_from_pdf_pymupdf(pdf_file)
        
        # Inicializar el modelo de embeddings y el almacenaje de vectores Chroma
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        
        # Crear el almacén de vectores Chroma
        vector_store = Chroma.from_texts(
            texts=pdf_lines,
            collection_name="pdf-input",
            embedding=embeddings,
        )

        # Crear el recuperador (retriever)
        retriever = vector_store.as_retriever()

        # Plantilla de Prompt
        template = """Answer the question based only on the following context:
        {context}

        Question: {question}
        """
        prompt = ChatPromptTemplate.from_template(template)

        # Modelo de chat con Ollama
        ollama_llm = "llama3.2"
        model_local = ChatOllama(model=ollama_llm)

        # Crear la cadena de procesamiento
        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | model_local
            | StrOutputParser()
        )

        # Ejecutar la cadena con la consulta
        response = chain.invoke(input)
        return response
    except Exception as e:
        return f"Error al procesar la consulta: {e}"

Tip:
 Always set type="messages" in gr.ChatInterface. The default value (type="tuples") is deprecated and will be removed in a future version of Gradio.

In [69]:
# Crear la interfaz con Gradio
with gr.Blocks(theme=gr.themes.Glass()) as demo:
    gr.Markdown("### Chat con modelo de análisis de PDF")

    # Sección para el chat
    chat_interface = gr.ChatInterface(
        fn=search_chroma,
        type="messages",
        examples=["¿Quién es Martin Scorsese?", "¿Qué relación tiene la violencia en sus películas?", "Háblame de Goodfellas"],
    )

    # Espacio para subir archivos
    with gr.Row():
        gr.Markdown("### Subir un archivo para analizar:")
        file_uploader = gr.File(label="Sube tu archivo aquí", file_types=[".pdf"])
        output = gr.Textbox(label="Salida", lines=10)
        submit_button = gr.Button("Procesar archivo")

    # Conectar el procesamiento del archivo con el chat
    def process_file(pdf_file):
        global pdf_lines
        pdf_lines = extract_lines_from_pdf_pymupdf(pdf_file)
        return "Archivo procesado. Puedes hacer preguntas ahora."

    # Conectar el botón de envío a la función de procesamiento
    submit_button.click(fn=process_file, inputs=file_uploader, outputs=output)

# Lanzar la interfaz
demo.launch()

* Running on local URL:  http://127.0.0.1:7892

To create a public link, set `share=True` in `launch()`.
